# <span style="color:green"><h1><center>Baseflow Separation</center></h1></span>
<center>Prepared by <br>
    <b>Venkatesh Merwade</b><br> 
School of Civil Engineering, Purdue University<br>
vmerwade@purdue.edu<br>
<br>
<b>Spring 2020<br>
    FAIR Science in Water Resources</b><br></center>

## <span style="color:green">Objective</span>
<p style='text-align: justify;'>The main objective of this tutorial is to estimate baseflow from an observed hydrograph at a river location so that direct (surface) runoff resulting from the excess rainfall can be separated. In case of a perennial or continuous flow regime (characteristic of humid climate), a nearly non varying flow occurs even in a rainless period of the year. This delayed flow occurs due to the infiltrated water from a previous rainfall event in the watershed eventually making its way to the river through subsurface flow and seepage through the channel bed. In order to study the effect of a current rainfall event, the baseflow component has to be separated from the total flow and direct runoff (total flow minus baseflow) has to be calculated. A typical runoff hydrograph representing the surface runoff and baseflow is shown below along with the associated rainfall and excess rainfall (effective rainfall).</p> 

<img src="https://www.researchgate.net/profile/Barbara_Lastoria/publication/228905789/figure/fig6/AS:667656462028801@1536193113095/Examples-of-simple-graphical-base-flow-separation-methods.png" alt="Drawing" style="width:369px" />
<center><b>Source</b>: Hydrological processes on the land surface: A survey of modelling approaches - Scientific Figure on ResearchGate.<br>Available from: https://www.researchgate.net/figure/Examples-of-simple-graphical-base-flow-separation-methods_fig6_228905789</center>

## <span style="color:green">Baseflow Separation Methodology</span>

<p style='text-align: justify;'>There are various methods developed to separate the baseflow from the total streamflow. The ground water can make up the substantial portion in a streamflow and its contribution must be evaluated.</p>

<span style="color:black">This program separate the baseflow from streamflow using two methods based on</span> 
<ul>
    <li><span style="color:red">Arnold et al (1999)</span></li>
    <li><span style="color:red">Eckhardt (2005)</span></li>
</ul>
    
<p style='text-align: justify;'>The <i>first method</i> used in this tutorial (Arnold et al (1995)) uses digital filter technique which was originally used in signal analysis and processing. The equation of the filter is given by:</p>
$$q_t = \beta  q_{t-1} + (1+\beta)/2 \times(Q_t - Q_{t-1})$$

where 
<ul>
    <li>$q_t$ - filtered surface runoff (quick response) at the t time step (one day)</li>
    <li>Q - original streamflow</li>
    <li>$\beta$ - filter parameter (0.925)</li>
    
The baseflow $b_t$ is given by:
$$b_t = Q_t - q_t$$

<p style='text-align: left;'>In this method, filters are passed over the streamflow data three times (forward, backward and forward) depending upon the the user's selected estimates of baseflow. The consequence of each pass is that it will reduce the baseflow as a percentage of total flow.</p>

<p style='text-align: left;'>The second method used in the tutorial is proposed by Eckhardt (2005). It is considered as a special separtion technique which involves recursive digital filtering of hydrographs. This method is centered on simple one-parameter filter ($\alpha$ is the parameter). The partitions the streamflow into two components, namely direct runoff and baseflow and is represented by:</p>
$$y_k = f_k + b_k$$

where 
<ul>
    <li>y - total streamflow</li>
    <li>f - direct runoff</li>
    <li>b - baseflow</li>
    <li>k - time step number</li>

The general form is expressed in terms of one parameter filter($\alpha$) and $BFI_{max}$. It is given by:

$$b_k = \frac{(1 - BFI_{max}) \times \alpha \times b_{k-1} + (1-\alpha) \times BFI_{max}\times y_k}{1 - \alpha \times BFI_{max}}$$

$\alpha$ varies between 0 and 1. $BFI_{max}$ which is the maximum value of baseflow index (long-term ratio of baseflow to total streamflow) is always less than 1. In this tutorial, we will consider $\alpha$ as 0.98 and $BFI_{max}$ as 0.8.

## Data Source

<p style='text-align: justify;'>For the manual calculation of baseflow separation, the daily streamflow can be obtained from the link: https://waterdata.usgs.gov/nwis/sw. In this tutorial, we use hydrofunctions python library to download streamflow for a USGS station.</p>

Overview of steps
<ol type="1">
    <span style="color:red"><li> Create the definition block for two different methods of baseflow separation.</li></span>
    <span style="color:red"><li>Obtain the daily streamflow data for a USGS gage station for a particular period.</li></span>
     <span style="color:red"><li>Delineate baseflow based on AR and EK methods</li></span>
     <span style="color:red"><li>Plot a discharge hydrograph indicating the total runoff and baseflow.</li></span>


## <span style="color:green">Import the packages/modules required for this exercise</span>

We need four packages: hydrofunctions (hf), pandas (pd), numpy (np), and pyplot (plt) from matplotlib. The paranthesis contains the commonly used short forms for these libraries.

In [ ]:
#WRITE THE CODE TO IMPORT THE ABOVE FUNCTIONS

## <span style="color:green">First Method for Baseflow Separation</span> 

<p style='text-align: justify;'>Let us define the first definition block in Python to implement the method suggested by <b><u> Arnold et al</u></b>. This definition block will be later invoked in the code.</p>
<ul>
<li>Input for the function: <span style="color:red">strflow - Pandas Series of Streamflow data [Continuous streamflow measurements at river gage]</span></li>
<li>Output of the function: <span style="color:red">BFlow - Numpy Array of Baseflow data [Baseflow or subsurface flow filtered by three passes]</span></li>
    

In [ ]:
def AR_baseflow(strflow):
    print ("Baseflow Separation Method 1 - Execution started")
    
    ## Intial conditions
    a = .925
    b = (1+a) / 2
    flow = np.array(strflow)
    DR = np.array(strflow)
    BFlow = np.zeros([len(DR),3])
    DR[0] = flow[0] * 0.5
    BFlow[0,0] = flow[0] - DR[0]
    BFlow[0,1] = BFlow[0,0]
    BFlow[0,2] = BFlow[0,0]
    # First pass [forward]
    for i in range(1,len(flow)):
        DR[i] = a * DR[i-1] + b * (flow[i] - flow[i-1])
        if (DR[i] < 0):
            DR[i] = 0
            
        BFlow[i,0] = flow[i] - DR[i]
        if (BFlow[i,0] < 0):
            BFlow[i,0] = 0
            
        if (BFlow[i,0] > flow[i]):
            BFlow[i,0] = flow[i]
    
    ## Second pass [backward]
    BFlow[len(flow)-1,1] = BFlow[len(flow)-1,0]
    for i in range(len(flow)-2,-1,-1):    
        DR[i] = a * DR[i+1] + b * (BFlow[i,0] - BFlow[i+1,0])
        if DR[i] < 0:
            DR[i] = 0
        BFlow[i,1] = BFlow[i,0] - DR[i]
        if BFlow[i,1] < 0:
            BFlow[i,1] = 0
        if BFlow[i,1] > BFlow[i,0]:
            BFlow[i,1] = BFlow[i,0]
    
    ## Third pass [forward]
    BFlow[len(flow)-1,2] = BFlow[len(flow)-1,0]
    for i in range(1,len(flow)):
        DR[i] = a * DR[i-1] + b * (BFlow[i,1]- BFlow[i-1,1])
        if DR[i] < 0:
            DR[i] = 0
        BFlow[i,2] = BFlow[i,1] - DR[i]
        if BFlow[i,2] < 0:
            BFlow[i,2] = 0
        if BFlow[i,2] > BFlow[i,1]:
            BFlow[i,2] = BFlow[i,1]
    print ("Baseflow Separation Method 1 - Execution completed successfully \n")
    return(BFlow[:,2])

## <span style="color:green">Second Method for Baseflow Separation</span>

Let us define the second definition block in Python to implement the method suggested by <b><u> Eckhardt</u></b>. This definition block will be later invoked in the code.
<ul>
<li> Input for the function: <span style="color:red">strflow  - Pandas Series of Streamflow data [Continuous streamflow measurements at river gage]</span></li>
<li> Output of the function: <span style="color:red">BFlow  - Numpy Array of Baseflow data [Baseflow or subsurface flow]</span></li>
    

In [ ]:
def EK_baseflow(strflow):
    
    print ("Baseflow Separation Method 2 - Execution started")
    
    alpha=0.98
    BFI_max = 0.8
    flow = np.array(strflow)
    BFlow = np.zeros([len(flow)])
    BFlow[0] = flow[0]
    for i in range(1,len(flow)):
    # algorithm
            BFlow[i] = ((1 - BFI_max) * alpha * BFlow[i-1] + (1 - alpha) * BFI_max * flow[i]) / (1 - alpha * BFI_max)
            if BFlow[i] > flow[i]:
                BFlow[i] = flow[i]
    print ("Baseflow Separation Method 2 - Execution completed successfully \n")
    return(BFlow)

## <span style="color:green">USGS Station and Analysis Period</span>

We need three inputs from the user for carrying out the analysis:
<ol type="1">
    <span style="color:red"><li> USGS station code</li></span>
    <span style="color:red"><li> Analysis start date </li></span>
    <span style="color:red"><li> Analysis end date </li></span>
    
<p style='text-align: justify;'><b><u> Note:</u></b> Both dates value should be entered in YYYY-MM-DD format (including the hyphen symbol). It would be better to select <b>ONE YEAR</b> period to get a better view of the hydrograph. Also, make sure that the USGS station code is a string.<br>
<br>
To check whether your code is working correcly, let us use  the USGS gage on Wabash River at Lafayette, IN (<b>03335500</b>) for a sample analysis period from <b>2017-01-01</b> to <b>2017-01-31</b>.</p>

In [ ]:
# WRITE A CODE TO DEFINE VARIABLES FOR STATION NUMBER,START DATE AND END DATE

## <span style="color:green">Obtain the data using hydrofunction from USGS NWIS</span>

One of the two approaches below may be used to obtain the daily value "dv" of the streamflow

#### <span style="color:red">Approach01</span>
<code>data = hf.NWIS(USGS_StationCode,'dv',period='P365D') # P365D indicates past 365 days</code>

#### <span style="color:red">Approach02</span>
This more useful as discharges for desired period can be obtained and let us use the approach in this tutorial. <br>
<code>data = hf.NWIS(USGS_StationCode, 'dv', Start_Date,End_Date)</code>

Finally, run <code>data.get_data()</code> to download the daily data from USGS.

In [ ]:
#WRITE THE CODE TO DOWNLOAD DAILY STREAMFLOW FROM USGS USING HYDROFUNCTION

## <span style="color:green">Data Checking</span>

<p style='text-align: justify;'>Now, let us check the data by printing the first n rows (default: 5) using the head statement. This is helpful for quickly testing if your object has the right type of data in it.</p>

<u><b>Note:</b></u>
    
00060 is parameter code for Discharge (cfs)<br>
00003 is the stat code for daily mean

In [ ]:
#WRITE YOUR CODE HERE
print(data.df().head())
print("\n")

## <span style="color:green">Dataframe and Column name change</span>

<p style='text-align: justify;'>We get the output from the hydrofunction as "class 'function'". By defining a new dataframe, let us store the data into pandas dataframe to do the manipulations. In the above data, we have two columns namely Discharge and Qualification Code. Further, we will drop the column containing the qualifiers. Also, we change the cloumn name of discharge from long name ("USGS:03335500:00060:00003") to short name ("Total_Runoff") for easy handling in the succeeding code.</p>


In [ ]:
strflow=pd.DataFrame(data.df())
strflow=strflow.drop(columns='USGS:03335500:00060:00003_qualifiers')
strflow=strflow.rename(columns={'USGS:03335500:00060:00003': 'Total_Runoff'})

## <span style="color:green">Discharge hydrograph (without baseflow)</span>


Now, we will plot the discharge hydrograph (without baseflow)

In [ ]:
%matplotlib inline
#WRITE YOUR CODE HERE TO PLOT THE DISCHARGE HYDROGRAPH
#ALSO PROVIDE AXIS TITLES WITH UNITS

## <span style="color:green">Baseflow Separation</span>

<p style='text-align: justify;'>So this is time to use the definition blocks which were earlier defined. We will use <span style="color:red"><i>AR_baseflow</i></span> and <span style="color:red"><i>EK_baseflow</i></span> functions to separate the baseflow from the total runoff. Further, we will print the simple sum to check whether our results are consistent (to ensure you have not missed out anything wrong). The mean value of two methods for Wabash River at Lafayette, IN (<b>03335500</b>) for a sample analysis period from <b>2017-01-01</b> to <b>2017-01-31</b> are <b>7121.417</b> $ft^3/s$ (AR_baseflow) and <b>10941.493</b> $ft^3/s$ (EK_baseflow).</p> 

In [ ]:
strflow['AR_baseflow'] = #WRITE YOUR OWN CODE
strflow['EK_baseflow'] = #WRITE YOUR OWN CODE
print(sum(strflow['AR_baseflow'])/len(strflow['AR_baseflow']))
print(sum(strflow['EK_baseflow'])/len(strflow['AR_baseflow']))

## <span style="color:green">Discharge hydrograph (with baseflow separated)</span>

Now, we will plot the discharge hydrograph showing the separated baseflows from two methods

In [ ]:
#WRITE YOUR CODE HERE TO PLOT THE DISCHARGE HYDROGRAPH WITH BASEFLOW SEPARATIONS
#ALSO PROVIDE AXIS TITLES WITH UNITS

<b>Ok, you have completed the tutorial!</b>

## <span style="color:green">Turn-in</span>

<ol type="1">
    <span style="color:black"><li>The discharge hydrograph showing the total runoff of particular USGS Station for atleast one year period. Mention clearly the Station Code and data period used that was used in the analysis.</li></span>
    <br>
    <span style="color:black"><li>The discharge hydrograph showing the total runoff and two baseflow curves obtained using the two different methods.</li></span>



## <span style="color:green">References</span>

<ul>
<p style='text-align: justify;'><li><span style="color:black">J.G. Arnold and P.M Allen. Automated methods for estimating BFlow and groundwater recharge from streamflow records. Journal of the Americam Water Resources Association vol 35(2) (April 1999): 411-424.</span></li></p>
<p style='text-align: justify;'><li><span style="color:black">J.G. Arnold, P.M. Allen, R. Muttiah, and G. Bernhardt, Automated base flow separation and recession analysis techniques. Ground Water vol 33(6): 1010-1018.</span></li></p>
<p style='text-align: justify;'><li><span style="color:black">Eckhardt, Klaus., How to construct recursive digital filters for baseflow separation, Hydrological Processes: An International Journal 19, no. 2 (2005): 507-515.</span></li></p>
    